# Damiano Bressanin 138075

# Fonti:
- https://keras.io/guides/transfer_learning/
- https://keras.io/api/applications/mobilenet/
- "MobileNetV2: Inverted Residuals and Linear Bottlenecks" https://arxiv.org/pdf/1801.04381.pdf



# Test:
- Dataset 80-10-10;
- Foto a colori;
- Sì Data Augmentation;
- Transfer Learning e Fine Tuning con MobileNetV2;

In [1]:
import os
import shutil
import random
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import numpy as np

from matplotlib.image import imread
from matplotlib import pyplot

random.seed(1)

2023-08-29 12:55:14.157305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing e Data Augmentation

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input



train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
#    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.8,1.2],
#    rotation_range=20,
#    width_shift_range=0.15,
#    height_shift_range=0.15,
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


batch_size = 64

train_it = train_datagen.flow_from_directory("dataset_diviso_v2/train/",
                                             class_mode='binary',
                                             batch_size=batch_size,
                                             target_size=(224, 224),
                                             color_mode="rgb",
                                             shuffle=True,
                                             seed=42
                                            )

val_it = val_datagen.flow_from_directory('dataset_diviso_v2/valid/',
                                         class_mode='binary',
                                         batch_size=batch_size,
                                         target_size=(224, 224),
                                         color_mode="rgb",
                                         shuffle=True,
                                         seed=42)

test_it = test_datagen.flow_from_directory('dataset_diviso_v2/test/',
                                           class_mode='binary',
                                           batch_size=batch_size,
                                           target_size=(224, 224),
                                           color_mode="rgb",
                                           shuffle=True,
                                           seed=42)



Found 19997 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


# Creazione del modello:
# Transfer Learning

In [3]:
from tensorflow.keras.applications import MobileNetV2
from keras.models import Model


def define_model():
    # carico il modello
    model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # congelo gli strati
    for layer in model.layers:
        layer.trainable = False
    # aggiungo parte finale
    flat1 = GlobalAveragePooling2D()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    class2 = Dense(64, activation='relu', kernel_initializer='he_uniform') (class1)
    output = Dense(1, activation='sigmoid')(class2)

    model = Model(inputs=model.inputs, outputs=output)

    #opt = SGD(learning_rate=0.0001, momentum=0.9)
    opt = Adam(learning_rate=0.0001)
    #opt = RMSprop(learning_rate=0.0001)
    
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [5]:
import datetime

unique_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + unique_name

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='epoch')

In [6]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

model=define_model()

history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=val_it, validation_steps=len(val_it), epochs=4, verbose=1, callbacks=[tensorboard_callback,early_stop],validation_split=0)

2023-08-29 12:55:18.814829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 12:55:18.822708: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 12:55:18.823311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 12:55:18.825173: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 12:55:18.825658: I tensorflow/compile

Epoch 1/4


2023-08-29 12:55:27.758025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-29 12:55:29.819580: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1a9da810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-29 12:55:29.819635: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-08-29 12:55:29.826974: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-29 12:55:29.992073: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 41/313 [==>...........................] - ETA: 3:21 - loss: 0.3310 - accuracy: 0.8754

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:866: UserWarning: Truncated File Read
  warnings.warn(str(msg))


313/313 [==============================] - 253s 785ms/step - loss: 0.0961 - accuracy: 0.9669 - val_loss: 0.0453 - val_accuracy: 0.9832
Epoch 2/4
313/313 [==============================] - 261s 833ms/step - loss: 0.0381 - accuracy: 0.9867 - val_loss: 0.0404 - val_accuracy: 0.9852
Epoch 3/4
313/313 [==============================] - 275s 877ms/step - loss: 0.0306 - accuracy: 0.9894 - val_loss: 0.0429 - val_accuracy: 0.9840
Epoch 4/4
313/313 [==============================] - 280s 894ms/step - loss: 0.0276 - accuracy: 0.9905 - val_loss: 0.0348 - val_accuracy: 0.9864


In [7]:
# valuto il modello

_, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
print('> %.3f' % (acc * 100.0))

40/40 [==============================] - 6s 138ms/step - loss: 0.0388 - accuracy: 0.9896
> 98.960


# Fine tuning

In [8]:
def trova_sottostringa(parola):
    if 'BN' in parola:
        return True
    else:
        return False



# voglio addestrare solo dal blocco X in poi

blockX_start_index = None
for index, layer in enumerate(model.layers):
    if layer.name == 'block_16_expand':
        blockX_start_index = index
        break

for layer in model.layers[:blockX_start_index]:
    layer.trainable = False

# lascio congelati i layer con la Batch Normalization

for layer in model.layers[blockX_start_index:]:
    if trova_sottostringa(layer.name)==False:
        #print(layer.name)
        layer.trainable = True
    else:
        #print(layer.name)
        layer.trainable = False



#for layer in model.layers:
#    layer.trainable = True


In [9]:

# learning rate più basso

opt=Adam(learning_rate=0.00001)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])



In [10]:
train_it.reset
val_it.reset
test_it.reset


history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=val_it, validation_steps=len(val_it), epochs=30, verbose=1, callbacks=[tensorboard_callback,early_stop],validation_split=0)


Epoch 1/30
313/313 [==============================] - 253s 790ms/step - loss: 0.0326 - accuracy: 0.9886 - val_loss: 0.0383 - val_accuracy: 0.9860
Epoch 2/30
313/313 [==============================] - 262s 837ms/step - loss: 0.0247 - accuracy: 0.9911 - val_loss: 0.0347 - val_accuracy: 0.9876
Epoch 3/30
313/313 [==============================] - 255s 816ms/step - loss: 0.0191 - accuracy: 0.9934 - val_loss: 0.0326 - val_accuracy: 0.9872
Epoch 4/30
313/313 [==============================] - 245s 782ms/step - loss: 0.0180 - accuracy: 0.9932 - val_loss: 0.0332 - val_accuracy: 0.9872
Epoch 5/30
313/313 [==============================] - 259s 828ms/step - loss: 0.0155 - accuracy: 0.9950 - val_loss: 0.0318 - val_accuracy: 0.9876
Epoch 6/30
313/313 [==============================] - 250s 799ms/step - loss: 0.0132 - accuracy: 0.9955 - val_loss: 0.0310 - val_accuracy: 0.9884
Epoch 7/30
313/313 [==============================] - 258s 823ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.0325 -

In [11]:
# valuto il modello

_, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
print('> %.3f' % (acc * 100.0))

40/40 [==============================] - 5s 127ms/step - loss: 0.0349 - accuracy: 0.9904
> 99.040


In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [13]:
model_dir = "modelli/"+ unique_name +".keras"
model.save(model_dir)

In [14]:
def summarize_diagnostics(history):
    pyplot.figure(figsize=(20, 15))
    
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='orange', label='train')
    pyplot.plot(history.history['val_loss'], color='blue', label='val')
    pyplot.legend(loc='upper right') # legenda
    pyplot.grid(which='both', linestyle='-', linewidth=1, color='black')  # griglia nera
    
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='red', label='train')
    pyplot.plot(history.history['val_accuracy'], color='green', label='val')
    pyplot.legend(loc='lower right')
    pyplot.grid(which='both', linestyle='-', linewidth=1, color='black')

    # salvo i plot su file
    filename = 'plot/' + unique_name
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()
    
summarize_diagnostics(history)